In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np
from string import punctuation

# Load Data

In [ ]:
!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/reviews.txt

--2020-02-06 18:12:09--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt.1’

labels.txt.1        100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2020-02-06 18:12:10 (4.30 MB/s) - ‘labels.txt.1’ saved [225000/225000]

--2020-02-06 18:12:11--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response...

# Load Data

In [ ]:
# read data from text files
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()

In [ ]:
print(reviews[:100])
print()
print(labels[:18])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life

positive
negative



# Data Preprocessing

In [ ]:
reviews = reviews.lower()

# remove punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])

In [ ]:
print(reviews[:100])
all_text[:100]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life


'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life '

In [ ]:
# split by new lines
reviews_split = all_text.split("\n")
all_text = " ".join(reviews_split)

words = all_text.split()

In [ ]:
print(reviews_split[:2])
print(all_text[:100])
print(words[:100])

['bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t   ', 'story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  viole

In [ ]:
type(reviews_split)

list

## Encode words

In [ ]:
# Encode words
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word:i for i, word in enumerate(vocab, 1)}

In [ ]:
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [ ]:
print('Unique words: ', len((vocab_to_int)))
print()

print('Tokenized review: \n', reviews_ints[0])

Unique words:  74072

Tokenized review: 
 [21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]


## Econde Labels

In [ ]:
labels_split = labels.split("\n")
encoded_labels = np.array([1 if label == "positive" else 0 for label in labels_split])

In [ ]:
encoded_labels[:10]

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

## Shape reviews
-> remove short reviews

In [ ]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [ ]:
sum(review_lens) / len(review_lens)

539.1170731707317

In [ ]:
print(len(reviews_ints))

non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

25001
Number of reviews after removing outliers:  25000


# Pad sequence

In [ ]:
# padding sequences
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), int)

    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]

    
    return features

In [ ]:
seq_length = 300 # avg - 536

features = pad_features(reviews_ints, seq_length)
print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

## Prepare Datasets

In [ ]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 300) 
Validation set: 	(2500, 300) 
Test set: 		(2500, 300)


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 300])
Sample input: 
 tensor([[   0,    0,    0,  ...,    3,  206,   66],
        [   0,    0,    0,  ...,    3,   45,    4],
        [   0,    0,    0,  ...,  512,  307, 6393],
        ...,
        [   0,    0,    0,  ...,   14,  281,  305],
        [  92,   57,    5,  ...,   59,  380,    8],
        [   0,    0,    0,  ...,    8,  123,  128]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 1])


In [ ]:
print("Batch size: ", train_loader.batch_size)
print("Train database shape: ", sample_x.shape)

for inputs, labels in train_loader:
  if inputs.shape != sample_x.shape:
    print("Faulty shape: ", inputs.shape)
    inp = inputs

Batch size:  50
Train database shape:  torch.Size([50, 300])


# Create Model

In [ ]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [ ]:
import pickle
f = open("vocab_to_int.pkl","wb")
pickle.dump(vocab_to_int,f)
f.close()

In [ ]:
data = pickle.load(open("vocab_to_int.pkl", 'rb'))

In [ ]:
type(data)

dict

In [ ]:
data['the']

1

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [ ]:
h = net.init_hidden(50)
net = net.cuda()
s, a = net(sample_x.cuda(), h)

In [ ]:
s.shape

torch.Size([50])

# Train

In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.714394... Val Loss: 0.651716
Epoch: 1/4... Step: 200... Loss: 0.685520... Val Loss: 0.691891
Epoch: 1/4... Step: 300... Loss: 0.651012... Val Loss: 0.674877
Epoch: 1/4... Step: 400... Loss: 0.673164... Val Loss: 0.574229
Epoch: 2/4... Step: 500... Loss: 0.522028... Val Loss: 0.507737
Epoch: 2/4... Step: 600... Loss: 0.492566... Val Loss: 0.497932
Epoch: 2/4... Step: 700... Loss: 0.405973... Val Loss: 0.525645
Epoch: 2/4... Step: 800... Loss: 0.497805... Val Loss: 0.582233
Epoch: 3/4... Step: 900... Loss: 0.266682... Val Loss: 0.434488
Epoch: 3/4... Step: 1000... Loss: 0.233062... Val Loss: 0.452182
Epoch: 3/4... Step: 1100... Loss: 0.417272... Val Loss: 0.394759
Epoch: 3/4... Step: 1200... Loss: 0.365560... Val Loss: 0.428575
Epoch: 4/4... Step: 1300... Loss: 0.068070... Val Loss: 0.480236
Epoch: 4/4... Step: 1400... Loss: 0.212902... Val Loss: 0.424719
Epoch: 4/4... Step: 1500... Loss: 0.525093... Val Loss: 0.523142
Epoch: 4/4... Step: 1600... Loss: 

# Test

In [ ]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.450
Test accuracy: 0.823


In [ ]:
torch.save(net.state_dict(), "model.pt")

In [ ]:
net.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

# Test with text

In [ ]:
import os


def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

In [ ]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

In [ ]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 

    i = output.item()
    print(i)

    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
        print("Positive percentage: {:.2f}%".format(i * 100))
    else:
        print("Negative review detected.")
        print("Negative percentage: {:.2f}%".format(100 - (i * 100)))

In [ ]:
test_review = 'I love this movie, this is the best ever seen in my life'
seq_length=200
predict(net, test_review, seq_length)

0.9751630425453186
Positive review detected!
Positive percentage: 97.52%


In [ ]:
test_review = 'I love this movie, this is the best ever seen in my life'
seq_length= 60
predict(net, test_review, seq_length)

Positive review detected!
Positive percentage: 97.75%


In [ ]:
test_review = 'I love this movie, this is the best ever seen in my life'
seq_length=len(test_review)
predict(net, test_review, seq_length)

Positive review detected!
Positive percentage: 97.76%


In [ ]:
test_review = 'This is a time killer movie, just wast of time'
seq_length=200
predict(net, test_review, seq_length)

Positive review detected!
Positive percentage: 95.14%


In [ ]:
torch.save(net.state_dict(), "model.pt")

# Prepare New Dataset

In [ ]:
def predictN(net, test_review, sequence_length=500):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 

    i = output.item()

    return i * 100

In [ ]:
seq_length=200
predictN(net, reviews_split[1], seq_length)

10.131609439849854

In [ ]:
labels_split[1]

'negative'

In [ ]:
def getClass(pred):
  s = ""
  if pred > 90:
    s = "Good"
  elif pred > 50:
    s = "Medium"
  elif pred > 10:
    s = "Average"
  elif pred > 0:
    s = "Poor"

  return s

In [ ]:
newClass = []

for i in reviews_split[:-1]:
  pre = predictN(net, i)
  cls = getClass(pre)
  newClass.append(cls)

In [ ]:
len(newClass), len(reviews_split[:-1]), len(labels_split[:-1])

(25000, 25000, 25000)

In [ ]:
dit = {
    "Review": reviews_split[:-1],
    "Label": labels_split[:-1],
    "Class": newClass
}

In [ ]:
import pandas as pd

df = pd.DataFrame(dit)

In [ ]:
df.head()

,Review,Label,Class
0,bromwell high is a cartoon comedy it ran at t...,positive,Good
1,story of a man who has unnatural feelings for ...,negative,Average
2,homelessness or houselessness as george carli...,positive,Average
3,airport starts as a brand new luxury pla...,negative,Poor
4,brilliant over acting by lesley ann warren b...,positive,Good


In [ ]:
df.Class.value_counts()

Poor       10417
Good        9657
Average     2587
Medium      2339
Name: Class, dtype: int64